In [ ]:
# install required modules
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langgraph
!pip install -q load_dotenv
!pip install -q langchain-community
!pip install -q faiss-cpu
!pip install -q -U langchain_community
!pip install -q -U langchain_pinecone
!pip install requests beautifulsoup4

In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 8.0 MB/s eta 0:00:00


In [ ]:
import os
from langchain_community.document_loaders import DirectoryLoader,PyPDFLoader,TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from google.colab import drive

#uncomment this when mounting the google drive to Colab for the first time
drive.mount('/content/gdrive')

    # --------------------------
    # 1. Load your Knowledge Base
    # --------------------------
    # Assume all your page content (e.g. text or HTML files) is stored under the 'documents' directory.
    # Adjust the file pattern as needed (e.g. "**/*.html" for HTML files).
#loader = DirectoryLoader("/content/gdrive/MyDrive/site_content/",  glob="**/*.txt", show_progress=True, loader_cls=TextLoader)
loader = DirectoryLoader("/content/gdrive/MyDrive/temp/",  glob="**/*.pdf", show_progress=True, loader_cls=PyPDFLoader)
    #loader = DirectoryLoader("documents", glob="**/*.txt")
print("Loading documents from the SIIG manual pdf filesknowledge base...")
documents = loader.load()
print(f"Loaded {len(documents)} documents.")

    # --------------------------
    # 2. Split Documents into Chunks
    # --------------------------
    # Splitting text into chunks helps the model process long documents and retrieve relevant pieces.
#text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
#docs = text_splitter.split_documents(documents)
#print(f"Split documents into {len(docs)} chunks.")

#separate lines by \n
text_splitter = CharacterTextSplitter(separator="\n",chunk_size=3000, chunk_overlap=100)
idocs = text_splitter.split_documents(documents)
docs = [x for x in idocs if (len(x.page_content) > 5 and not(x.page_content.startswith("URL:")) ) ]
print(f"Split documents into {len(docs)} chunks. {docs[0]}")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Loading documents from the SIIG manual pdf filesknowledge base...



  1%|▏         | 26/1974 [09:42<12:06:58, 22.39s/it]

100%|██████████| 1/1 [00:01<00:00,  1.35s/it]

Loaded 1 documents.


IndexError: list index out of range

In [ ]:
import os
from langchain_community.document_loaders import DirectoryLoader,PyPDFLoader,TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from google.colab import drive

#uncomment this when mounting the google drive to Colab for the first time
drive.mount('/content/gdrive')

    # --------------------------
    # 1. Load your Knowledge Base
    # --------------------------
    # Assume all your page content (e.g. text or HTML files) is stored under the 'documents' directory.
    # Adjust the file pattern as needed (e.g. "**/*.html" for HTML files).
loader = DirectoryLoader("/content/gdrive/MyDrive/product_data_7_9/",  glob="**/*.txt", show_progress=True, loader_cls=TextLoader)
    #loader = DirectoryLoader("documents", glob="**/*.txt")
print("Loading documents from the SIIG manual pdf filesknowledge base...")
documents = loader.load()
print(f"Loaded {len(documents)} documents.")

    # --------------------------
    # 2. Split Documents into Chunks
    # --------------------------
    # Splitting text into chunks helps the model process long documents and retrieve relevant pieces.
#text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
#docs = text_splitter.split_documents(documents)
#print(f"Split documents into {len(docs)} chunks.")

#separate lines by \n
#text_splitter = CharacterTextSplitter(separator="\n",chunk_size=1000, chunk_overlap=100)
#idocs = text_splitter.split_documents(documents)
docs = [x for x in documents if (len(x.page_content) > 5 and len(x.page_content) <= 30000 and not(x.page_content.startswith("URL:")) ) ]
print(f"Split documents into {len(docs)} chunks.")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Loading documents from the SIIG manual pdf filesknowledge base...



100%|██████████| 1/1 [00:00<00:00,  2.27it/s]

Loaded 1 documents.
Split documents into 1 chunks.


In [ ]:
print(docs[0])

page_content='﻿Below is a detailed comparison of SIIG’s industrial charging products based on the four model numbers provided. While the exact datasheets should be consulted for final verification, the following summary outlines key specifications, design features, and intended use cases for each model:
________________


Overview of Models
1. ID-US0A11-S1
 – Type: 600W 16‑port industrial charging station
– Total Power: Approximately 600 W
– Per-Port Output: Up to ~30 W
– Charging Protocols: USB‑C Power Delivery (PD 2.0/3.0)
– Target Devices: Smartphones, tablets, and other low‑power USB‑C devices
– Design & Build:
 • Rugged, metal chassis with rack‑mountable design
• Integrated active (fan‑assisted) cooling system
• Basic LED indicators and a master power switch
– Use Case: Ideal for environments (e.g., schools, IT labs, offices) where many devices need simultaneous charging without high‑power laptop requirements

2. ID-US0C11-S1
 – Type: 100W 16‑port industrial charging station
Speci

In [ ]:
from langchain_pinecone import PineconeVectorStore
os.environ["OPENAI_API_KEY"] = "????"
os.environ['PINECONE_API_KEY'] = '????'
index_name = "manuals"
embeddings = OpenAIEmbeddings()
vectorstore= PineconeVectorStore.from_documents(
    docs,
    index_name=index_name,
    embedding=embeddings
)
query = "Can ID-US0C11-S1 support 60W PD charging"
result = vectorstore.similarity_search(query)
print(result)




In [ ]:
import re

docstring = "CE-MT3N11-S1 is a replacement for CE-MT2611-S1"
print(re.findall("\w\w-\w+-\w\w", docstring))


['CE-MT3N11-S1', 'CE-MT2611-S1']


In [ ]:
!pip install -q -U tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.6 MB/s eta 0:00:00


In [ ]:
# To install, run: pip install tavily-python
from tavily import TavilyClient

client = TavilyClient(api_key="????")

response = client.search(
    query="how much is ID-US0C11-S1?"
)

print(response)


{'query': 'how much is ID-US0C11-S1?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': '16-Port Industrial USB-C PD Charging Station - 1000W - SIIG', 'url': 'https://www.siig.com/16-port-industrial-usb-c-pd-charging-station-1000w.html', 'content': 'ID-US0C11-S1. $599.99. Add to Wish List Add to Compare. Email. Join our free Membership Program and save 5% off every order. We ship all orders within 1 business day. Contact us now for more information about our products. Lifetime technical support on all of our products. 45-day return window and money-back guarantee.', 'score': 0.8815438, 'raw_content': None}, {'title': 'SIIG 16-Port Industrial USB-C PD Charging Station (ID-US0C11-S1) - eBay', 'url': 'https://www.ebay.com/itm/135286266608', 'content': "SIIG 16-Port Industrial USB-C PD Charging Station (ID-US0C11-S1) e-TekDepot (4132) 99.6% positive; Seller's other items Seller's other items; Contact seller; US $322.89. or Best Offer. Was US $339.88 (5% off)

In [ ]:
print(response['results'][0]['content'])
print(len(response['results'][0]['content']))

Mfr #: CE-MT3N11-S1. UNSPSC #: 43211602. 0. Your price: USD $77.99. In stock: 45 available. View similar items. Select quantity. Add to cart. Tech overview. SIIG Security Lockable Countertop Kiosk Stand Holder for iPad - Kiosk - for tablet - stand - lockable - metal, aluminum - white - screen size: 9.7"-11" - counter top ... Total price: USD
343


In [ ]:

from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings.openai import OpenAIEmbeddings
from google.colab import userdata
import os
import re
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
#text_field = "text"
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')

#client = Pinecone(api_key=PINECONE_API_KEY)

#for idx  in client.list_indexes():
#    print(idx)
index_name = "manuals"
embeddings = OpenAIEmbeddings()
vectorstore = PineconeVectorStore(
    index_name=index_name, embedding=embeddings,  pinecone_api_key=PINECONE_API_KEY
)
print("Vector store created.")
query = "what is CE-H28A11-S1"
result = vectorstore.similarity_search(query, k=15)
for doc in result:
  print(doc.metadata)
  print(doc.id)
  if len(doc.page_content) == 0 or (len(doc.page_content) < 20 and (re.search("\w\w-\w+-\w\w", doc.page_content) is None)) or( re.search("Comments:\n________", doc.page_content) is not None)  :
    if ("siig_manuals_txt" in doc.metadata['source']) :
      print("delete!!!!!-------",doc.page_content)
      #vectorstore.delete(ids=[doc.id])
  else :
    if ("siig_manuals_txt" in doc.metadata['source']) :
      print("delete!!!!!-------",doc.page_content)
      #vectorstore.delete(ids=[doc.id])


#uncomment this to add more docs to the vectorstore
#vectorstore.add_documents(documents=docs)
#print(docs[0])

#vectorstore.delete(ids=['ebb3239e-18af-440e-98e4-eb025cf6e70a'])



<ipython-input-2-8d9d0c697896>:22: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


Vector store created.
{'source': '/content/gdrive/MyDrive/site_content/products_it-products_kvms_extender_hdmi-2-0-kvm-over-cat6-extender-with-loopout-s-pdif.html.txt'}
87641efe-24d0-47bc-af9c-982e1ccc5e4f
{'source': '/content/gdrive/MyDrive/product_data/CE-H24H11-S1.txt'}
dec0672c-3253-425e-9a47-b35999634c0d
{'source': '/content/gdrive/MyDrive/site_content/av-products_pro-av_1x2-8k-hdmi-splitter-with-edid-audio-extractor.html.txt'}
ee7232d9-1943-44d2-84e5-fa519abc81c4
{'source': '/content/gdrive/MyDrive/site_content/taa-compliant_avpro_1x2-8k-hdmi-splitter-with-edid-audio-extractor.html.txt'}
b4367817-eb08-4761-9b0e-f48e2fe2bf81
{'source': '/content/gdrive/MyDrive/site_content/1x2-8k-hdmi-splitter-with-edid-audio-extractor.html.txt'}
5d87b175-7763-40db-b9c0-e554c7ba0c43
{'source': '/content/gdrive/MyDrive/product_data/CE-H24E11-S1.txt'}
056497c2-dd96-49b7-8947-fd3f45d37d75
{'source': '/content/gdrive/MyDrive/product_data/CE-H20211-S1.txt'}
81b57285-e15f-43cb-92e0-225a20231270
{'source

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
context = ""
with open("/content/gdrive/MyDrive/temp/CE-H23A11-S2.txt", 'r') as f:
  context = f.read()

In [ ]:
!pip install crawl4ai

In [ ]:
#test scrape 1 URL and parse the text
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
import os

# Please verify that scraping https://help.sumologic.com/ complies with its robots.txt and terms of service before running.
myheaders = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
url = "https://www.siig.com/search/CE-H28A11-S1"
if(1 > 0) :
    try:
        print(f"Fetching {url}", myheaders)
        response = requests.get(url,headers=myheaders)
        print(response)
        if response.status_code != 200:
            print(f"Non-200 status code {response.status_code} for URL: {url}")
    except Exception as e:
        print(f"Error fetching {url}: {e}")
    soup = BeautifulSoup(response.content, "html.parser")
    # Get visible text; you might want to customize this extraction.
    #context = soup.get_text(separator="\n", strip=True)
    context = soup.find('main').text.strip()
    lines = context.splitlines()
    processed_lines = [re.sub(r"\s+", " ", line).strip() for line in lines]
    processed_lines = filter(lambda x: len(x) > 0, processed_lines)
    context =  "\n".join(processed_lines)
    print(len(context))

Fetching https://www.siig.com/search/CE-H28A11-S1 {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
<Response [200]>
16655


In [ ]:
print(context)

In [ ]:
from langchain_core.tools import tool

# Simple web search tool
@tool
def web_search(query : str):
    """search web with query."""
    url = f"https://www.google.com/search?q={query}"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.text, 'html.parser')
    results = []

    for g in soup.find_all('div', class_='tF2Cxc')[:3]:  # top 3 results
        title = g.find('h3').text
        link = g.find('a')['href']
        snippet = g.find('span', class_='aCOpRe').text if g.find('span', class_='aCOpRe') else ""

        results.append({'title': title, 'link': link, 'snippet': snippet})

    return results

# Function definition for OpenAI API
function_tools = [
    {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Perform a web search and get the top 3 results.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The search query"}
                },
                "required": ["query"]
            }
        }
    }
]


In [ ]:
#Test AI chatbot server side
import requests
from bs4 import BeautifulSoup
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.chains import create_history_aware_retriever
import re

product_pages = {}

def retrieve_page(part_num : str) :
  #check if we already have this part number already
  if part_num in product_pages : return product_pages[part_num]

  myheaders = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
  url = "https://www.siig.com/search/" + part_num
  if(1 > 0) :
    try:
        print(f"Fetching {url}", myheaders)
        response = requests.get(url,headers=myheaders)
        print(response)
        if response.status_code != 200:
            print(f"Non-200 status code {response.status_code} for URL: {url}")
    except Exception as e:
        print(f"Error fetching {url}: {e}")
    soup = BeautifulSoup(response.content, "html.parser")
    # Get visible text; you might want to customize this extraction.
    #context = soup.get_text(separator="\n", strip=True)
    pcontent = soup.find('main').text.strip()
    lines = pcontent.splitlines()
    processed_lines = [re.sub(r"\s+", " ", line).strip() for line in lines]
    processed_lines = filter(lambda x: len(x) > 0, processed_lines)
    pcontent =  "\n".join(processed_lines)
    #only keep the first 6000 characters
    print(len(pcontent))
    if len(pcontent) > 6000 :
      pcontent = pcontent[:6000]

    product_pages[part_num] = pcontent
    return pcontent

def get_additional_context(qqq : str) :
  part_nums = re.findall("\w\w-\w+-\w\w", qqq)
  more_contexts = ""
  part_nums = list(set(part_nums))
  for part_num in part_nums:
    page = retrieve_page(part_num)
    more_contexts = f"SKU or part number : {part_num} , " + page
  return more_contexts


if (1>0):
    # create history aware retriever and qa_chain
    #llm = ChatOpenAI(model="gpt-4o", temperature=0,openai_api_key=userdata.get('OPENAI_API_KEY') )
    llm = ChatOpenAI(model="gpt-4o", temperature=0,openai_api_key=userdata.get('OPENAI_API_KEY'),tools =function_tools,tool_choice="auto")
    #tools = [web_search]
    #llm_with_tools = llm.bind_tools(tools)
    #msg1 = llm_with_tools.invoke("what is the difference between CE-H23A11-S2 and CE-H23B11-S2")
    #print(msg1.tool_calls)
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 15})

    contextualize_q_system_prompt = """Given a chat history and the latest user question \
    which might reference context in the chat history, formulate a standalone question \
    which can be understood without the chat history. Do NOT answer the question, \
    just reformulate it if needed and otherwise return it as is."""
    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )

    #If you don't know the answer, try using the web_search tool provided to search for info on the web, and use the search results as the context to respond
    textual_description_of_tool="""
    web_search(query: str) :
        Perform a web search and get the top 3 results.

        Args:
            query: The search query (string).

        Returns:
            A list of strings containing the search result snippets
    """
    qa_system_prompt = """Do not make up answer! \
    You are a professional customer service agent working for SIIG. Always be polite, clear, and helpful.\
    Use the following pieces of retrieved context to answer the question. \
    if you answer contains 'emailing us at [email protected]', replace it with 'email us at support@siig.com'\
    if you answer has an SIIG address that contains '6078 Stewart Avenue',  use this address instead - \
    'SIIG, 31038 Huntwood Ave., Hayward, CA 94544' \
    if you don't know the anwser, try to use the provided tool web_search to find more context \
    after you try all possible means, you still don't know the answer, say please visit our website https://siig.com or contact support at support@siig.com . \
    Try to keep the answer very accurate,  and if possible, make it concise.\
    {context}"""

    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", qa_system_prompt),
            MessagesPlaceholder("additional_context"),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )

    print("qa_prompt = ", qa_prompt)
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    qa_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    chat_history = []

    while True:
      query = input("\nYour question: ")
      if query.lower() in ["exit", "quit"]:
        print("Goodbye!")
        break
      try:
        #print(qa_chain)
        print(query)
        chat_history_string = " ".join([f"{msg}" for msg in chat_history])
        #print(chat_history_string)
        additional = get_additional_context(query+chat_history_string)

        print("-------------------")
        print(llm.invoke("summarize these descriptions to less than 300 words :"+additional))
        print("-------------------")
        #print(additional)
        #answer = qa_chain.run(query)
        #chat_history = get_session_history(sessionid)
        ai_msg_1 =qa_chain.invoke({"input": query, "chat_history": chat_history,"additional_context":[SystemMessage(additional)]})

        print(ai_msg_1)
        print(ai_msg_1.get("tool_calls"))
        #print(ai_msg_1["answer"])
        chat_history.extend([HumanMessage(content=query), ai_msg_1["answer"]])
        #update_session(sessionid,chat_history)
        answer = ai_msg_1["answer"]
        #print("\nchat history:",  chat_history)
        print("\nAnswer:", answer)
      except Exception as e:
        answer = "Sorry, an error occurred:"+str(e)
        print("\nAnswer:", answer)

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: WARNING! tools is not default parameter.
                tools was transferred to model_kwargs.
                Please confirm that tools is what you intended.
  if (await self.run_code(code, result,  async_=asy)):
/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: WARNING! tool_choice is not default parameter.
                tool_choice was transferred to model_kwargs.
                Please confirm that tool_choice is what you intended.
  if (await self.run_code(code, result,  async_=asy)):


qa_prompt =  input_variables=['additional_context', 'chat_history', 'context', 'input'] input_types={'additional_context': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMes

In [ ]:
harryp_qa_dict = {}
harryp_qa_dict['question'] = "what is the color"
harryp_qa_dict['answers'] = ["red","blue","white"]
harryp_qa_dict['probabilitioes'] = []
harryp_qa_dict['question2'] = "what is the shape"
harryp_qa_dict['answer2'] = "round"


total_p =0
for i in (8) :

  # randomly pick a question form list of 20 questions

  # add to a set

  # text to speechl, ask user

  # record natural language response

  # speech to text -> store in user_input

  # give context/prompt (chosen question, answer, probabilities) + user_input + append "give me the answer choice"

  # llm returns the choice

  # calculate probability based on the answer choice - find the probability from the choice



In [ ]:
dual_context = get_additional_context("what is the difference between CE-H23A11-S2 and CE-H23B11-S2")
print(dual_context)


In [ ]:
#test gpt4o only without RAG

query = "what is the difference between CE-H23A11-S2 and CE-H23B11-S2"
llm = ChatOpenAI(model="gpt-4o", temperature=0,openai_api_key=userdata.get('OPENAI_API_KEY'))

chat_history = []
system_prompt = (
    "Context: {chat_history}"
)
while True:
  query = input("\nYour question: ")
  if query.lower() in ["exit", "quit"]:
    print("Goodbye!")
    break
  try:
    messages =  [
        ("system", context),
        ("human", query),
      ]

    answer = llm.invoke(messages)
    print("\nAnswer:", answer)
    chat_history.extend([HumanMessage(content=query), answer])
  except Exception as e:
    print("Sorry, an error occurred:", e)

In [ ]:
!pip install -q google-search-results

  Preparing metadata (setup.py) ... done


In [ ]:
#test google search with serpapi
from serpapi import GoogleSearch

params = {
  "api_key": "???",
  "engine": "google",
  "q": "what is the difference between ID-US0B11-S3 and ID-US0C11-S1",
  "google_domain": "google.com",
  "gl": "us",
  "hl": "en"
}

search = GoogleSearch(params)
results = search.get_dict()

In [ ]:
print(results['organic_results'][0])

{'position': 1, 'title': 'Power Charging - Power - IT PRODUCTS', 'link': 'https://www.siig.com/products/it-products/power/power-charging.html?srsltid=AfmBOoqwYjBdgNmzxCqXB25GAz_yxNWOxPN8D2OOM8A4iSzYzcotv0KT', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.siig.com/products/it-products/power/power-charging.html%3Fsrsltid%3DAfmBOoqwYjBdgNmzxCqXB25GAz_yxNWOxPN8D2OOM8A4iSzYzcotv0KT&ved=2ahUKEwihpqnLhdaLAxVKna8BHZgDIzQQFnoECBgQAQ', 'displayed_link': 'https://www.siig.com › products › power-charging', 'favicon': 'https://serpapi.com/searches/67b91ce863c076c360aadf7e/images/a326a0677a12a970c867494fef4ca48b1f160b9ff56c7dad459585443e2476e0.png', 'snippet': 'ID-US0B11-S3. Industrial Grade USB-C 5Gbps Hub & PD Charging Station - 600W, Adds 16 USB-C Charging Ports, 2-port 100W & 14-port Charging ...', 'snippet_highlighted_words': ['Industrial Grade USB-C 5Gbps Hub & PD Charging Station'], 'source': 'SIIG, Inc.'}


In [ ]:
from langchain.chains import RetrievalQA
# --------------------------
# 4. Create a Retriever and the QA Chain
# --------------------------
# The retriever will use similarity search (e.g., k=5 nearest chunks) to get relevant context.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 20})

# Initialize the LLM (here, using OpenAI with a temperature of 0 for deterministic output)
llm = ChatOpenAI(model="gpt-4o", temperature=0,openai_api_key="???")
print(retriever)

system_prompt = (
    "You are a professional customer service agent. Always be polite, clear, and helpful."
    "Use the given context to answer the question. "
    "If you don't know the answer, try to find it online. "
    "Try to Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
#query = "Can ID-US0C11-S1 support 60W PD charging"
#question_answer_chain = create_stuff_documents_chain(llm, prompt)
#chain = create_retrieval_chain(retriever, question_answer_chain)
#chain.invoke({"input": query})

# Build a RetrievalQA chain: this chain will retrieve relevant document chunks and then pass them
# as context to the LLM to generate an answer.
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
answer = qa_chain.invoke({"query":"describe AC-PW1X11-S1"})
print(answer)

tags=['PineconeVectorStore', 'OpenAIEmbeddings'] vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x787ba4483e90> search_kwargs={'k': 20}
{'query': 'describe AC-PW1X11-S1', 'result': 'The AC-PW1X11-S1 is a powerful 4-port high-power charger designed to charge multiple devices simultaneously. It features three USB-C ports and one USB-A port, supporting a maximum total output of 240W. The charger is compatible with PD 3.1 or PD 3.0, Fast Charger 2.0, PPS, and Quick Charge 3.0 fast charging technologies. It is ideal for charging laptops, tablets, smartphones, and smartwatches. The charger is equipped with GaN 2 technology, which enhances charging efficiency and reduces heat. It includes a stand, carrying pouch, and quick start guide in the package. The charger is engineered in the USA and manufactured in China, and it comes with a 2-year limited warranty.'}


In [ ]:
from langchain_openai import ChatOpenAI

def main():
    # Set your OpenAI API key (ensure this key is kept secret!)
    os.environ["OPENAI_API_KEY"] = "???"  # Replace with your key or set it as an environment variable


    # --------------------------
    # 3. Create the Vector Store with Embeddings
    # --------------------------
    # We use OpenAI's embeddings to index the document chunks and store them in a FAISS index.
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(docs, embeddings)
    print("Vector store created.")

    # --------------------------
    # 4. Create a Retriever and the QA Chain
    # --------------------------
    # The retriever will use similarity search (e.g., k=5 nearest chunks) to get relevant context.
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

    # Initialize the LLM (here, using OpenAI with a temperature of 0 for deterministic output)
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0,openai_api_key="???")

    # Build a RetrievalQA chain: this chain will retrieve relevant document chunks and then pass them
    # as context to the LLM to generate an answer.
    qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

    # --------------------------
    # 5. Chat Loop for Product Support
    # --------------------------
    print("Welcome to the Product Support Chatbot!")
    print("Ask your question about our products or type 'exit' to quit.")
    while True:
        query = input("\nYour question: ")
        if query.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break
        try:
            answer = qa_chain.run(query)
            print("\nAnswer:", answer)
        except Exception as e:
            print("Sorry, an error occurred:", e)

if __name__ == "__main__":
    main()



Vector store created.
Welcome to the Product Support Chatbot!
Ask your question about our products or type 'exit' to quit.

Your question: who are you?

Answer: I'm an AI language model created by OpenAI, here to help answer your questions and provide information. How can I assist you today?

Your question: for CE-MT3H11-S1, does it have gas spring?

Answer: I don't have specific information on the model CE-MT3H11-S1 in the provided documents. Therefore, I cannot confirm whether it includes a gas spring or not. If you need details about this specific model, I recommend checking the product manual or the manufacturer's website for accurate information.

Your question: for ID-US0C11-S1, does it support 60W PD charging?

Answer: Yes, the ID-US0C11-S1 supports 60W PD charging when all 16 USB-C ports are in use.

Your question: what is its total charging power?

Answer: The total charging power of the 16-port industrial USB-C PD Charging Station is 1000W.

Your question: does the JU-DK0P11-

In [ ]:
import pandas as pd
from pandas import DataFrame
from google.colab import drive

def save_product(product) :
  #print(product['sku'])
  file_name = product['sku'] + ".txt"
  with open(f"/content/gdrive/MyDrive/data/{file_name}", 'w') as f:
    for key, value in product.items():
      f.write(f"{key}: {value} \n")



#uncomment this when mounting the google drive to Colab for the first time
drive.mount('/content/gdrive')

df_product = pd.read_excel("/content/gdrive/MyDrive/data/siig-products-1.xlsx", sheet_name="catalog_product_entity")
df_attr = pd.read_excel("/content/gdrive/MyDrive/data/siig_website_attribute.xlsx", sheet_name="siig_website_attribute")


#df_attr.head(5)
df_product = pd.merge(df_product, df_attr[['attribute_id','attribute_code']], left_on='attribute_id', right_on='attribute_id', how='left').drop('attribute_id', axis=1)
#df_product.head(5)

df_product.dropna(subset=['sku'], inplace=True)

product = {}
current_sku = ''
for index, row in df_product.iterrows():
  if current_sku != row['sku'] :
    current_sku = row['sku']
    if product != {}:
      if product['sku'] != 'nan': save_product(product)
    product = {}

  product['sku'] = row['sku']
  product[row['attribute_code']] = row['value']




In [ ]:

from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/chat', methods=['POST'])
def chat():
    user_message = request.json.get('message')
    # Process the user message with your chatbot logic
    response_message = your_chatbot_function(user_message)
    return jsonify({'reply': response_message})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)